In [1]:
from logger import Logger
logger = Logger()
logger.info("テスト")

テスト


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [3]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
         else "cpu")
logger.info(f"Using {device} device")

Using mps device


In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [5]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [6]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork()

In [19]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [30]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), batch*batch_size + len(X)
            logger.info(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [31]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    logger.info(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
len(train_dataloader)

938

In [24]:
len(train_dataloader.dataset)

60000

In [26]:
%%time
# CPU使用
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

loss: 0.790894 [   64/60000]


Epoch 1
-------------------------------


loss: 0.877226 [ 6464/60000]
loss: 0.640411 [12864/60000]
loss: 0.842191 [19264/60000]
loss: 0.735839 [25664/60000]
loss: 0.725993 [32064/60000]
loss: 0.820958 [38464/60000]
loss: 0.787480 [44864/60000]
loss: 0.812493 [51264/60000]
loss: 0.766125 [57664/60000]
Test Error: 
 Accuracy: 72.4%, Avg loss: 0.761808 

loss: 0.752257 [   64/60000]


Epoch 2
-------------------------------


loss: 0.846199 [ 6464/60000]
loss: 0.607155 [12864/60000]
loss: 0.817317 [19264/60000]
loss: 0.714183 [25664/60000]
loss: 0.700010 [32064/60000]
loss: 0.795633 [38464/60000]
loss: 0.769631 [44864/60000]
loss: 0.789738 [51264/60000]
loss: 0.744558 [57664/60000]
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.738886 

loss: 0.718732 [   64/60000]


...

Epoch 9
-------------------------------


loss: 0.690992 [ 6464/60000]
loss: 0.469100 [12864/60000]
loss: 0.711222 [19264/60000]
loss: 0.624421 [25664/60000]
loss: 0.606633 [32064/60000]
loss: 0.664848 [38464/60000]
loss: 0.693433 [44864/60000]
loss: 0.695008 [51264/60000]
loss: 0.639118 [57664/60000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.630448 

loss: 0.568146 [   64/60000]


Epoch 10
-------------------------------


loss: 0.675287 [ 6464/60000]
loss: 0.456822 [12864/60000]
loss: 0.700873 [19264/60000]
loss: 0.616523 [25664/60000]
loss: 0.599540 [32064/60000]
loss: 0.651098 [38464/60000]
loss: 0.687541 [44864/60000]
loss: 0.687445 [51264/60000]
loss: 0.627878 [57664/60000]
Test Error: 
 Accuracy: 78.5%, Avg loss: 0.619680 



Done!
CPU times: user 1min 31s, sys: 6.68 s, total: 1min 38s
Wall time: 25.8 s


In [33]:
model = NeuralNetwork().to(device)

In [34]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [35]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [36]:
%%time
# GPU使用?

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

loss: 2.300111 [   64/60000]


Epoch 1
-------------------------------


loss: 2.289084 [ 6464/60000]
loss: 2.268006 [12864/60000]
loss: 2.255447 [19264/60000]
loss: 2.238539 [25664/60000]
loss: 2.204539 [32064/60000]
loss: 2.221730 [38464/60000]
loss: 2.182754 [44864/60000]
loss: 2.175158 [51264/60000]
loss: 2.145915 [57664/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 2.138950 

loss: 2.151685 [   64/60000]


Epoch 2
-------------------------------


loss: 2.146328 [ 6464/60000]
loss: 2.087251 [12864/60000]
loss: 2.097001 [19264/60000]
loss: 2.039102 [25664/60000]
loss: 1.981200 [32064/60000]
loss: 2.012524 [38464/60000]
loss: 1.933543 [44864/60000]
loss: 1.933668 [51264/60000]
loss: 1.858321 [57664/60000]
Test Error: 
 Accuracy: 59.6%, Avg loss: 1.860419 

loss: 1.895909 [   64/60000]


...

Epoch 9
-------------------------------


loss: 0.951391 [ 6464/60000]
loss: 0.713793 [12864/60000]
loss: 0.903064 [19264/60000]
loss: 0.794348 [25664/60000]
loss: 0.795807 [32064/60000]
loss: 0.880052 [38464/60000]
loss: 0.837634 [44864/60000]
loss: 0.846820 [51264/60000]
loss: 0.821505 [57664/60000]
Test Error: 
 Accuracy: 69.5%, Avg loss: 0.814692 

loss: 0.835588 [   64/60000]


Epoch 10
-------------------------------


loss: 0.913664 [ 6464/60000]
loss: 0.669732 [12864/60000]
loss: 0.867636 [19264/60000]
loss: 0.764484 [25664/60000]
loss: 0.758057 [32064/60000]
loss: 0.849325 [38464/60000]
loss: 0.814843 [44864/60000]
loss: 0.816822 [51264/60000]
loss: 0.794512 [57664/60000]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.785475 



Done!
CPU times: user 26.8 s, sys: 2.06 s, total: 28.8 s
Wall time: 33 s
